In [1]:
import os
from os import path
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
from datetime import datetime
date = datetime.today().strftime('%y%m%d')
print ('Last modified: ' + date)

Last modified: 210802


In [3]:
df = pd.read_csv('Technology Use (OA; pilot)_May 28, 2021_14.40.csv', encoding = "ISO-8859-1")
df = df.drop(df.index[[0,1]]) #drop top rows

In [4]:
cols = df.columns.drop('ResponseId')
df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')

# Computer proficiency

In [5]:
names = df.columns[pd.Series(df.columns).str.startswith('comp_')]
df['cpq'] = df[names].sum(axis=1)

# Mobile proficiency

In [6]:
names = df.columns[pd.Series(df.columns).str.startswith('mobile_')]
df['mdpq'] = df[names].sum(axis=1)

# Wifi proficiency

In [7]:
names = df.columns[pd.Series(df.columns).str.startswith('wifi_')]
df['wnpq'] = df[names].sum(axis=1)


# All proficiency

In [8]:
df['all_proficiency'] = df['cpq'] + df['mdpq'] + df['wnpq']

# Hardware

In [9]:
names = df.columns[pd.Series(df.columns).str.startswith('hardware')]
df['hardware'] = df[names].sum(axis=1)

# Software

In [10]:
names = df.columns[pd.Series(df.columns).str.startswith('software')]
df['software'] = df[names].sum(axis=1)
df['software']

2     16
3     20
4     21
5     13
6     13
7     20
8     19
9     21
10    16
11    18
12    18
13    15
14    17
15    14
16    16
17    14
18    13
19    19
20    14
21    18
22    17
23    20
24    20
25    17
26    21
27    19
28    19
29    17
30    14
31    15
Name: software, dtype: int64

# Recall

In [11]:
df = df.apply(lambda x: x.str.lower() if x.dtype=='object' else x) #convert ALL string columns to lowercase

In [12]:
correct = ['book','child','gold','hotel','king','market','paper','river','skin','tree']
imm_columns = ['imm_1', 'imm_2', 'imm_3', 'imm_4', 'imm_5', 'imm_6','imm_7', 'imm_8', 'imm_9', 'imm_10']
delay_columns = ['delay_1', 'delay_2', 'delay_3', 'delay_4', 'delay_5', 'delay_6','delay_7', 'delay_8', 'delay_9', 'delay_10']

## Immediate recall

In [13]:
df_transposed = df[imm_columns].T #transpose the df

a = list()
#keep only unique recall answers
for i in df_transposed.columns:
    a.append(df_transposed[i].unique())
    
# recall scoring rules
# for each cell, verify if a word is one of the ten; if yes, add 1 to the score
b = [0]*len(a)
x = 0
for i in a:
    for j in i:
        if j in correct: 
            b[x] = b[x] + 1
    x = x + 1
    
df['imm_sum'] = b

## Delayed recall

In [14]:
df_transposed = df[delay_columns].T #transpose the df

a = list()
#keep only unique recall answers
for i in df_transposed.columns:
    a.append(df_transposed[i].unique())
    
# recall scoring rules
# for each cell, verify if a word is one of the ten; if yes, add 1 to the score
b = [0]*len(a)
x = 0
for i in a:
    for j in i:
        if j in correct: 
            b[x] = b[x] + 1
    x = x + 1
    
df['delay_sum'] = b

In [15]:
df['delay_sum']

2     10
3      7
4      2
5      5
6      5
7      5
8      6
9      6
10     4
11     5
12     2
13     1
14     5
15     6
16     6
17     5
18     3
19     3
20     1
21     4
22     5
23     4
24     7
25     3
26     6
27     4
28     3
29     7
30     3
31     4
Name: delay_sum, dtype: int64

# Sense of control

In [16]:
a = ['control_1', 'control_2', 'control_4', 'control_5', 'control_7', 'control_9', 'control_10','control_11']
b = ['control_3', 'control_6', 'control_8','control_12']

In [17]:
df['constraints'] = df[a].sum(axis=1)
df['mastery'] = df[b].sum(axis=1)


# Exercise

In [18]:
df['exer_sum'] = df['exer_1'] + 2*df['exer_2'] + 3*df['exer_3']

# Social encouragement

In [19]:
names = df.columns[pd.Series(df.columns).str.startswith('social_')]
df['social'] = df[names].sum(axis=1)

In [20]:
df['social']

2      2
3      6
4      4
5      6
6      1
7      4
8      7
9      8
10     0
11     6
12     0
13     4
14     7
15     6
16     2
17     0
18    10
19     0
20     4
21     8
22     6
23     6
24    10
25    10
26     3
27     2
28     0
29     7
30     6
31     0
Name: social, dtype: int64

In [21]:
df1 = df.rename(columns={"Duration (in seconds)": "duration"})

In [22]:
subset = ['ResponseId', 'cpq','mdpq','wnpq','all_proficiency','hardware','software',
          'imm_sum','delay_sum','constraints','mastery','exer_sum', 'social',
          'map','number','volunteer','dentist',
          'checking','retirement','work','vehicle','inheritance_1','debts',
          'age','sage','education','gender','hispanic','english_1','race','couple',
          'shealth','hearing','vision','alcohol','effort','experience','duration']

df1 = df1[subset]

# Save 

In [23]:
df1.to_csv(('prolific_pilot_'+ date + '.csv'),index = False)